In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.utils import shuffle

# Charger les données
data = pd.read_csv('listings.csv')

# Traitement des données manquantes sans utiliser inplace=True
data['last_review'] = data['last_review'].fillna(data['last_review'].mode()[0])
data['reviews_per_month'] = data['reviews_per_month'].fillna(data['reviews_per_month'].median())

# Sélection des variables sans inclure la colonne 'neighbourhood_group'
features = ['latitude', 'longitude', 'room_type', 'minimum_nights', 'number_of_reviews', 'last_review', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']
target = 'price'

X = data[features]
y = np.log1p(data[target])  # Transformation log du prix

# Diviser les données en train et test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Prétraitement des données
numeric_features = ['latitude', 'longitude', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']
categorical_features = ['room_type', 'last_review']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Modèle Bagging avec DecisionTreeRegressor
bagging_regressor = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', BaggingRegressor(n_estimators=100, random_state=42))
])

# Validation croisée (score MAE)
cross_val_scores = cross_val_score(bagging_regressor, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')

# Affichage des résultats de validation croisée
print("Validation croisée - MAE (Mean Absolute Error) :")
print("Scores de validation croisée (MAE) :", -cross_val_scores)  # On prend l'inverse du score car il est retourné négatif
print("Moyenne des scores : {:.2f}".format(-cross_val_scores.mean()))
print("Écart type des scores : {:.2f}".format(cross_val_scores.std()))

# Entraînement du modèle sur l'ensemble des données d'entraînement
bagging_regressor.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = bagging_regressor.predict(X_test)

# Affichage des métriques d'évaluation
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)  # Calcul du RMSE manuellement
r2 = r2_score(y_test, y_pred)

print("\nÉvaluation sur l'ensemble de test :")
print(f"MAE (Mean Absolute Error) : {mae:.2f}")
print(f"MSE (Mean Squared Error) : {mse:.2f}")
print(f"RMSE (Root Mean Squared Error) : {rmse:.2f}")
print(f"R² (Coefficient de détermination) : {r2:.2f}")


Validation croisée - MAE (Mean Absolute Error) :
Scores de validation croisée (MAE) : [0.37637073 0.37145021 0.37967323 0.37211201 0.37441318]
Moyenne des scores : 0.37
Écart type des scores : 0.00

Évaluation sur l'ensemble de test :
MAE (Mean Absolute Error) : 0.34
MSE (Mean Squared Error) : 0.24
RMSE (Root Mean Squared Error) : 0.49
R² (Coefficient de détermination) : 0.56


Le modèle de régression utilisé repose sur une méthode d’agrégation par Bagging avec un estimateur de base de type DecisionTreeRegressor.
Il a été évalué à l’aide d’une validation croisée sur l’ensemble d’apprentissage et d’une évaluation finale sur un jeu de test indépendant.


 Validation croisée (5 folds)
MAE moyen : 0.37

Écart-type : 0.00

Cela indique que les erreurs de prédiction sont très stables sur les différentes sous-parties de l’ensemble d’entraînement
ce qui reflète une bonne robustesse du modèle.

 Évaluation sur le jeu de test
MAE : 0.34

MSE : 0.24

RMSE : 0.49

R² : 0.56

Le modèle atteint une précision légèrement meilleure sur le jeu de test que pendant la validation croisée. 
Il explique 56 % de la variance du logarithme du prix (log1p(price)), 
ce qui est acceptable dans un contexte de données réelles comportant du bruit et des facteurs non observés (qualité du logement, saisonnalité, avis clients, etc.).



 Interprétation
 
 
Le MAE de 0.34 sur le logarithme du prix correspond à une erreur absolue moyenne d’environ 40 % en échelle réelle (selon la formule exp(MAE) - 1 ≈ 0.4)
ce qui reste raisonnable dans un contexte immobilier à forte variabilité.

Le R² de 0.56 suggère qu’il reste encore 44 % de variance non expliquée, ce qui peut venir :

de variables non incluses dans le modèle,

d une non-linéarité complexe entre les variables ou de bruit intrinsèque dans les données.



Limites identifiées




Utilisation du log(price) :

Si cela stabilise les extrêmes, cela rend les erreurs moins lisibles pour une interprétation métier. Il faudrait retransformer (np.expm1) les prédictions pour les comparer en euros.

Variables non exploitables :

La variable last_review est actuellement traitée comme catégorielle via OneHot, ce qui n’a pas vraiment de sens temporellement. Elle pourrait être remplacée par :

un nombre de jours depuis la dernière review,

ou convertie en année/mois.

Peu d interactions entre les variables :

Le modèle de régression par arbres individuels ne capture pas toujours efficacement les interactions complexes entre variables.

Pas de sélection de caractéristiques :

Toutes les variables disponibles ont été utilisées sans évaluation de leur pertinence réelle.




 Axes d amélioration
 
 
 
Remodeler la variable last_review :

La transformer en durée depuis la dernière review (en jours), ce qui est une information temporelle plus utile que son label brut.

Tester d autres modèles plus puissants :

RandomForestRegressor, GradientBoostingRegressor ou XGBoost peuvent offrir de meilleures performances.

Comparer les performances avec et sans Bagging.

Ajouter des variables dérivées :

Exemple : log(minimum_nights), reviews_per_month * number_of_reviews, ou des ratios personnalisés.

Utiliser la sélection de caractéristiques :

Par exemple, avec SelectKBest, RFE, ou des méthodes intégrées au modèle.

Évaluer les performances avec price (valeurs réelles) :

Retransformer les prédictions avec np.expm1(y_pred) pour évaluer en unités monétaires et non en log.


Conclusion
Le modèle bagging a permis d’obtenir une prédiction robuste et généralisable avec une bonne stabilité (écart-type faible);
des résultats raisonnables (MAE de 0.34 et R² de 0.56). 
Toutefois, des marges d amélioration existent sur le choix des variables, leur traitement, et le type de modèle. 
Cela ouvre la voie à une optimisation plus poussée dans les prochaines itérations.